In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
%cd gdrive/MyDrive/authorship_anaysis/authorship_analysis/training_models

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/MyDrive/authorship_anaysis/authorship_analysis/training_models


In [ ]:
! pip install -r ../requrements.txt
! python -m spacy download pl_core_news_lg

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached https://github.com/kpu/kenlm/archive/master.zip (553 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
2023-05-31 09:58:33.947788: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-31 09:58:35.104458: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-05-31 09:58:36.540041: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://gith

In [ ]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
print(module_path)
print(sys.path)

/content/gdrive/MyDrive/authorship_anaysis/authorship_analysis
['/content', '/env/python', '/usr/lib/python310.zip', '/usr/lib/python3.10', '/usr/lib/python3.10/lib-dynload', '', '/usr/local/lib/python3.10/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.10/dist-packages/IPython/extensions', '/root/.ipython', '/content/gdrive/MyDrive/authorship_anaysis/authorship_analysis']


In [ ]:
from data_preparation import CorpusPreparation
from pipelines import Pipeline, Explain
from data_fetchers import EpochsFetcher, BookSet

/usr/local/lib/python3.10/dist-packages/keras/backend.py:452: UserWarning: `tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
  warnings.warn(


In [ ]:
bookset = BookSet()
bookset.fetch()
kinds = ['Epika']
if kinds[0]=='Epika':
  authors = ['Eliza Orzeszkowa', 'Henryk Sienkiewicz', 'Bolesław Prus']
  books_list = bookset.list_of_books_to_author_title_list(bookset.get_books_by_authors_list_kinds(authors, kinds))
  books_epoch_list = [books_list]
if kinds[0]=='Liryka':
  authors = ['Daniel Naborowski', 'Mikołaj Sęp Szarzyński', 'Elżbieta Drużbacka', 'Adam Mickiewicz', 'Juliusz Słowacki', 'Cyprian Kamil Norwid', 'Adam Asnyk', 'Maria Konopnicka', 'Kazimierz Przerwa-Tetmajer', 'Bolesław Leśmian', 'Jan Kasprowicz']
  authors_b = ['Daniel Naborowski', 'Mikołaj Sęp Szarzyński', 'Elżbieta Drużbacka']
  authors_r = ['Adam Mickiewicz', 'Juliusz Słowacki', 'Cyprian Kamil Norwid']
  authors_p = ['Adam Asnyk', 'Maria Konopnicka']
  authors_m = ['Kazimierz Przerwa-Tetmajer', 'Bolesław Leśmian', 'Jan Kasprowicz']
  books_list_b = bookset.list_of_books_to_author_title_list(bookset.get_books_by_authors_list_kinds(authors_b, kinds))
  books_list_r = bookset.list_of_books_to_author_title_list(bookset.get_books_by_authors_list_kinds(authors_r, kinds))
  books_list_p = bookset.list_of_books_to_author_title_list(bookset.get_books_by_authors_list_kinds(authors_p, kinds))
  books_list_m = bookset.list_of_books_to_author_title_list(bookset.get_books_by_authors_list_kinds(authors_m, kinds))
  books_epoch_list = [books_list_b, books_list_r, books_list_p, books_list_m]
print(books_epoch_list)

[[('Bolesław Prus', 'Antek'), ('Eliza Orzeszkowa', 'Australczyk'), ('Eliza Orzeszkowa', 'Autobiografia w listach'), ('Henryk Sienkiewicz', 'Bartek zwycięzca'), ('Henryk Sienkiewicz', 'Bez dogmatu'), ('Eliza Orzeszkowa', 'Cicho, cicho na leśnej polanie...'), ('Eliza Orzeszkowa', 'Cnotliwi'), ('Eliza Orzeszkowa', 'Co mówił stary klon?'), ('Eliza Orzeszkowa', 'Czy pamiętasz?'), ('Eliza Orzeszkowa', 'Dobra pani'), ('Eliza Orzeszkowa', 'Dwa bieguny'), ('Eliza Orzeszkowa', 'Dymy'), ('Eliza Orzeszkowa', 'Dziurdziowie'), ('Eliza Orzeszkowa', 'Šunadvokatis'), ('Bolesław Prus', 'Emancypantki'), ('Bolesław Prus', 'Emancypantki, tom pierwszy'), ('Bolesław Prus', 'Emancypantki, tom drugi'), ('Bolesław Prus', 'Faraon'), ('Bolesław Prus', 'Faraon, tom drugi'), ('Bolesław Prus', 'Faraon, tom pierwszy'), ('Bolesław Prus', 'Faraon, tom trzeci'), ('Eliza Orzeszkowa', 'Gdzie szczęście?'), ('Eliza Orzeszkowa', 'Gedali'), ('Eliza Orzeszkowa', 'Bóg wie kto'), ('Eliza Orzeszkowa', 'Dziwna historia'), ('Eliza 

In [ ]:
from os.path import exists
import pickle
datasets_filepath = 'datasets/features_'
for i in range(100):
  path = datasets_filepath + str(i)
  if exists(path):
    print(path)
    with open(path, 'rb') as f:
      data = pickle.load(f)
      head_data, _, _, _, _ = data
      print(f'test_size: {head_data[0]}, train_size: {head_data[1]}, number of words in paragraph: {head_data[2]}, preprocessing operations: {head_data[3]}, authors: {head_data[4]}')

datasets/features_0
test_size: 20, train_size: 100, number of words in paragraph: 15, preprocessing operations: ['anonymize'], authors: ['Daniel Naborowski', 'Mikołaj Sęp Szarzyński', 'Elżbieta Drużbacka', 'Adam Mickiewicz', 'Juliusz Słowacki', 'Cyprian Kamil Norwid', 'Adam Asnyk', 'Maria Konopnicka', 'Kazimierz Przerwa-Tetmajer', 'Bolesław Leśmian', 'Jan Kasprowicz']
datasets/features_1
test_size: 20, train_size: 100, number of words in paragraph: 15, preprocessing operations: ['anonymize', 'lower_text', 'remove_stop_words'], authors: ['Daniel Naborowski', 'Mikołaj Sęp Szarzyński', 'Elżbieta Drużbacka', 'Adam Mickiewicz', 'Juliusz Słowacki', 'Cyprian Kamil Norwid', 'Adam Asnyk', 'Maria Konopnicka', 'Kazimierz Przerwa-Tetmajer', 'Bolesław Leśmian', 'Jan Kasprowicz']
datasets/features_2
test_size: 20, train_size: 100, number of words in paragraph: 15, preprocessing operations: ['anonymize'], authors: ['Daniel Naborowski', 'Mikołaj Sęp Szarzyński', 'Elżbieta Drużbacka', 'Adam Mickiewicz'

In [ ]:
number_of_dataset = 30

In [ ]:
# file_name_read = datasets_filepath + str(number_of_dataset)
# with open(file_name_read, 'rb') as f:
#   data = pickle.load(f)
#   head_data, train_set, train_classes, test_set, test_classes = data
#   test_size, train_size, words_num_in_par, preprocessing_list, authors = head_data


train_size = 20
words_num_in_par = 300
test_size = 2
cp = CorpusPreparation(books_epoch_list)
cp.get_books()
cp.split_books(words_num_in_par)
train_set, train_classes, test_set, test_classes = cp.prepare_corpus_divide_books(train_size, test_size)

In [ ]:
existing_authors_set = set(train_classes)
authors_max_books = {}
authors = []
for books_epoch in books_epoch_list:
    authors_list = [author for (author, title) in books_epoch]
    authors_set = sorted(list(set(authors_list)))
    for author in authors_set:
        if author in existing_authors_set:
            books_num = authors_list.count(author)
            if author in authors:
                if books_num > authors_max_books[author]:
                    authors.remove(author)
                    authors_max_books[author] = books_num
                    authors.append(author)
            else:
                authors.append(author)
                authors_max_books[author] = books_num
authors


['Bolesław Prus', 'Eliza Orzeszkowa', 'Henryk Sienkiewicz']

In [ ]:
def get_samples_of_each_author(n: int):
  for i in range(len(authors)):
    a = train_classes[i*train_size]
    texts = train_set[i*train_size+5:i*train_size+n+5]
    with open('gpt_e.txt', 'a', encoding='utf-8') as f:
      f.write(f'Oto fragmenty tekstów {a}:\n\n')
      for t in texts:
        f.write('"')
        f.write(t)
        f.write('"')
        f.write('\n\n')
      f.write('\n\n\n')

In [ ]:
true_author = 'Henryk Sienkiewicz'
#true_author = 'Adam Mickiewicz'
#true_author = 'Juliusz Słowacki'
if true_author == 'Henryk Sienkiewicz':
  lime_text_path = './text_examples/sienkiewicz.txt'
if true_author == 'Adam Mickiewicz':
  lime_text_path = './text_examples/mickiewicz.txt'
if true_author == 'Juliusz Słowacki':
  lime_text_path = './text_examples/slowacki.txt'
with open(lime_text_path, encoding='utf-8') as f:
  text = f.read()
print(len(text.split()))
print(text)

703
Łaskawa Pani. 
Wyznaję otwarcie, że przed kilku laty, po otrzymaniu odpowiedzi Pani na mój pierwszy list, pomyślałem, żem się omylił albo nie w porę trafił i że te wszelkie niepospolite dary, które mogły się złożyć w Pani na wysoce harmonijną całość, znajdowały się przynajmniej w owej chwili, w stanie rozstroju. Odpowiedź Pani nie mogła mnie zachęcić do dalszego badania, jak było, a Jej następne podróże odjęły mi po temu możność. Uważałem, że na tej jednorazowej wymianie listów trzeba poprzestać z tej prostej przyczyny, że Pani tak sobie życzy, i wróciwszy do zwykłych obowiązków życia, starałem się więcej o tym nie myśleć. Po upływie kilku lat nie mogę też absolutnie twierdzić, czym kiedy z kim o Pani nie mówił - ale pozwalam sobie o tym wątpić, albowiem nieskłonny jestem do wynurzeń i prawie nigdy o osobistych rzeczach nie rozmawiam. 

Wyznaję również, że odpowiedź, jaką niegdyś otrzymałem na mój pierwszy list, sprawiła mi przykrość. Od pierwszej chwili poznania Pani uczułem dla N

In [ ]:
expl = Explain('')
pos = ['ADJ', 'NOUN', 'VERB']
preprocessing_list = ['anonymize']#, 'lower_text', 'remove_stop_words']#,'POS_leave_only']#, 'lemmatize_text', 'remove_punctuation']
text = expl.text_preprocessing(text, preprocessing_list, pos)
print(text)
print(len(text.split()))

Łaskawa Pani. 
Wyznaję otwarcie, że przed kilku laty, po otrzymaniu odpowiedzi Pani na mój pierwszy list, pomyślałem, żem się omylił albo nie w porę trafił i że te wszelkie niepospolite dary, które mogły się złożyć w Pani na wysoce harmonijną całość, znajdowały się przynajmniej w owej chwili, w stanie rozstroju. Odpowiedź Pani nie mogła mnie zachęcić do dalszego badania, jak było, a Jej następne podróże odjęły mi po temu możność. Uważałem, że na tej jednorazowej wymianie listów trzeba poprzestać z tej prostej przyczyny, że Pani tak sobie życzy, i wróciwszy do zwykłych obowiązków życia, starałem się więcej o tym nie myśleć. Po upływie kilku lat nie mogę też absolutnie twierdzić, czym kiedy z kim o Pani nie mówił - ale pozwalam sobie o tym wątpić, albowiem nieskłonny jestem do wynurzeń i prawie nigdy o osobistych rzeczach nie rozmawiam. 

Wyznaję również, że odpowiedź, jaką niegdyś otrzymałem na mój pierwszy list, sprawiła mi przykrość. Od pierwszej chwili poznania Pani uczułem dla Niej 

In [ ]:
get_samples_of_each_author(10)